***Import Required Libraries***

In [35]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

***Set Up Dataset Directory Structure***

*This section prepares the folder structure for training by merging all class images into a unified training folder grouped by label.*

In [38]:
base_dir = 'Data'
defective_dir = os.path.join(base_dir, 'Defective_bottle')
proper_dir = os.path.join(base_dir, 'Proper_bottle')
cans_dir = os.path.join(base_dir, 'train_cans')

unified_train_dir = 'UnifiedTrain'
shutil.rmtree(unified_train_dir, ignore_errors=True)
os.makedirs(unified_train_dir, exist_ok=True)
os.makedirs(os.path.join(unified_train_dir, 'Defective'), exist_ok=True)
os.makedirs(os.path.join(unified_train_dir, 'Proper'), exist_ok=True)
os.makedirs(os.path.join(unified_train_dir, 'Can'), exist_ok=True)

***Copy Images into Unified Directory***

*This function copies images from the original folders into the appropriate subdirectories within UnifiedTrain. It ensures that the dataset is correctly structured for the ImageDataGenerator.*|

In [43]:
def copy_images(src_dir, dst_dir):
    for img_name in os.listdir(src_dir):
        src_path = os.path.join(src_dir, img_name)
        if os.path.isfile(src_path):
            shutil.copy(src_path, dst_dir)

copy_images(defective_dir, os.path.join(unified_train_dir, 'Defective'))
copy_images(proper_dir, os.path.join(unified_train_dir, 'Proper'))
copy_images(cans_dir, os.path.join(unified_train_dir, 'Can'))


***Create Data Generators for Training and Validation***

*Here we create an ImageDataGenerator that normalizes image pixel values and splits the dataset into training and validation subsets.*



In [47]:
image_size = (224, 224)
batch_size = 32
train_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = train_gen.flow_from_directory(
    unified_train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = train_gen.flow_from_directory(
    unified_train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 770 images belonging to 3 classes.
Found 192 images belonging to 3 classes.


***Build the Transfer Learning Model with MobileNetV2***

*We initialize MobileNetV2 without its top layers and freeze its weights. Then, we stack new classification layers to adapt it for our 3-class bottle classification task.*

In [51]:
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


***Train the Model with Callbacks***

*Train the model using training data and monitor validation accuracy.
 Save the best model and stop early if accuracy stops improving*

In [55]:
checkpoint = ModelCheckpoint('bottle_can_classifier.h5', monitor='val_accuracy', save_best_only=True)
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=[checkpoint, early_stop]
)

C:\Users\nafiu\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step - accuracy: 0.3595 - loss: 1.3588

25/25 ━━━━━━━━━━━━━━━━━━━━ 14s 473ms/step - accuracy: 0.3612 - loss: 1.3536 - val_accuracy: 0.5052 - val_loss: 0.8794
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step - accuracy: 0.6386 - loss: 0.8366

25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 331ms/step - accuracy: 0.6402 - loss: 0.8341 - val_accuracy: 0.8646 - val_loss: 0.5531
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step - accuracy: 0.7954 - loss: 0.5778

25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 332ms/step - accuracy: 0.7961 - loss: 0.5763 - val_accuracy: 0.8906 - val_loss: 0.4080
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step - accuracy: 0.8444 - loss: 0.4302

25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 338ms/step - accuracy: 0.8452 - loss: 0.4300 - val_accuracy: 0.9219 - val_loss: 0.3430
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step - accuracy: 0.8696 - loss: 0.3706

25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 331ms/step - accuracy: 0.8693 - loss: 0.3707 - val_accuracy: 0.9375 - val_loss: 0.2909
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 322ms/step - accuracy: 0.9116 - loss: 0.2868 - val_accuracy: 0.9375 - val_loss: 0.2544
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step - accuracy: 0.9133 - loss: 0.2881

25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 340ms/step - accuracy: 0.9135 - loss: 0.2874 - val_accuracy: 0.9479 - val_loss: 0.2287
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 333ms/step - accuracy: 0.9079 - loss: 0.2581 - val_accuracy: 0.9479 - val_loss: 0.2120
Epoch 9/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step - accuracy: 0.9144 - loss: 0.2566

25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 331ms/step - accuracy: 0.9143 - loss: 0.2567 - val_accuracy: 0.9583 - val_loss: 0.1985
Epoch 10/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step - accuracy: 0.9304 - loss: 0.2347

25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 330ms/step - accuracy: 0.9302 - loss: 0.2342 - val_accuracy: 0.9635 - val_loss: 0.1927
Epoch 11/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 329ms/step - accuracy: 0.9305 - loss: 0.1989 - val_accuracy: 0.9427 - val_loss: 0.1783
Epoch 12/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 326ms/step - accuracy: 0.9348 - loss: 0.1846 - val_accuracy: 0.9479 - val_loss: 0.1811
Epoch 13/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 325ms/step - accuracy: 0.9257 - loss: 0.1829 - val_accuracy: 0.9427 - val_loss: 0.1745
Epoch 14/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 334ms/step - accuracy: 0.9592 - loss: 0.1443 - val_accuracy: 0.9583 - val_loss: 0.1653
Epoch 15/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 426ms/step - accuracy: 0.9450 - loss: 0.1554 - val_accuracy: 0.9427 - val_loss: 0.1644


***Evaluate the Model***

In [58]:
loss, accuracy = model.evaluate(val_data)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 296ms/step - accuracy: 0.9711 - loss: 0.1739
Validation Loss: 0.19271768629550934
Validation Accuracy: 0.9635416865348816
